In [1]:
import io
import boto3
import sagemaker
import pandas as pd
import matplotlib.pyplot as plt

sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()
bucket = sagemaker_session.default_bucket()
region = boto3.Session().region_name

sm = boto3.Session().client(service_name="sagemaker", region_name=region)

In [2]:
s3_client = boto3.client("s3")
BUCKET='ads508project'
KEY='Athena_Data/Source_Files/Ethnicity_Over_18.csv'
response = s3_client.get_object(Bucket=BUCKET, Key=KEY)
Ethnicity = pd.read_csv(response.get("Body"))

Ethnicity_Transformed = Ethnicity[["GEOID", "U7E001", "U7E002", "U7E003"]]
Ethnicity_Transformed = Ethnicity_Transformed.rename(columns={"GEOID":"GEO_ID", "U7E001":"TOTAL_ETHNICTY_18_OVER", "U7E002":"HISP_LATINO_18_OVER", "U7E003":"NOT_HISP_LATINO_18_OVER"}) 

Ethnicity_Transformed.head()

,GEO_ID,TOTAL_ETHNICTY_18_OVER,HISP_LATINO_18_OVER,NOT_HISP_LATINO_18_OVER
0,1500000US060014001001,1540,92,1448
1,1500000US060014001002,1093,57,1036
2,1500000US060014002001,832,74,758
3,1500000US060014002002,820,73,747
4,1500000US060014003001,914,62,852


In [3]:
s3_client = boto3.client("s3")
BUCKET='ads508project'
KEY='Athena_Data/Ethnicity_Over_18/Ethnicity_Over_18.csv'
response = s3_client.get_object(Bucket=BUCKET, Key=KEY)

with io.StringIO() as csv_buffer:
    Ethnicity_Transformed.to_csv(csv_buffer, index=False)

    response = s3_client.put_object(
        Bucket=BUCKET, Key=KEY, Body=csv_buffer.getvalue()
    )

    status = response.get("ResponseMetadata", {}).get("HTTPStatusCode")

    if status == 200:
        print(f"Successful S3 put_object response. Status - {status}")
    else:
        print(f"Unsuccessful S3 put_object response. Status - {status}")

Successful S3 put_object response. Status - 200


In [4]:
Ethnicity_data_types = Ethnicity_Transformed.dtypes
print(Ethnicity_data_types)
print('Number of Rows:', Ethnicity_Transformed.shape[0])

GEO_ID                     object
TOTAL_ETHNICTY_18_OVER      int64
HISP_LATINO_18_OVER         int64
NOT_HISP_LATINO_18_OVER     int64
dtype: object
Number of Rows: 25607


In [5]:
stats_hisp = pd.DataFrame(Ethnicity_Transformed['HISP_LATINO_18_OVER'].describe()).T
stats_hisp

,count,mean,std,min,25%,50%,75%,max
HISP_LATINO_18_OVER,25607.0,432.822822,377.566406,0.0,144.0,312.0,634.0,8851.0


In [6]:
stats_not_hisp = pd.DataFrame(Ethnicity_Transformed['NOT_HISP_LATINO_18_OVER'].describe()).T
stats_not_hisp

,count,mean,std,min,25%,50%,75%,max
NOT_HISP_LATINO_18_OVER,25607.0,771.031788,495.194364,0.0,452.0,704.0,1020.0,22429.0


In [7]:
s3_client = boto3.client("s3")
BUCKET='ads508project'
KEY='Athena_Data/Source_Files/Race_Over_18.csv'
response = s3_client.get_object(Bucket=BUCKET, Key=KEY)
Race = pd.read_csv(response.get("Body"))

Race_Transformed = Race[["GEOID", "U7D002", "U7D003", "U7D004", "U7D005", "U7D006", "U7D007", "U7D008"]]
Race_Transformed = Race_Transformed.rename(columns={"GEOID":"GEO_ID" ,"U7D002":"TOTAL_ONE_RACE_18_OVER", "U7D003":"WHITE_ALONE_18_OVER", "U7D004":"BLACK_ALONE_18_OVER", "U7D005":"NATIVE_AM_ALONE_18_OVER", "U7D006":"ASIAN_ALONE_18_OVER", "U7D007":"HAWAIIN_PAC_ISLAND_ALONE_18_OVER", "U7D008":"OTHER_ALONE_18_OVER"})

Race_Transformed.head()

,GEO_ID,TOTAL_ONE_RACE_18_OVER,WHITE_ALONE_18_OVER,BLACK_ALONE_18_OVER,NATIVE_AM_ALONE_18_OVER,ASIAN_ALONE_18_OVER,HAWAIIN_PAC_ISLAND_ALONE_18_OVER,OTHER_ALONE_18_OVER
0,1500000US060014001001,1377,1067,42,2,223,6,37
1,1500000US060014001002,996,638,77,0,265,2,14
2,1500000US060014002001,754,642,12,0,74,2,24
3,1500000US060014002002,735,580,27,0,114,0,14
4,1500000US060014003001,838,683,29,2,110,0,14


In [8]:
s3_client = boto3.client("s3")
BUCKET='ads508project'
KEY='Athena_Data/Race_Over_18/Race_Over_18.csv'
response = s3_client.get_object(Bucket=BUCKET, Key=KEY)

with io.StringIO() as csv_buffer:
    Race_Transformed.to_csv(csv_buffer, index=False)

    response = s3_client.put_object(
        Bucket=BUCKET, Key=KEY, Body=csv_buffer.getvalue()
    )

    status = response.get("ResponseMetadata", {}).get("HTTPStatusCode")

    if status == 200:
        print(f"Successful S3 put_object response. Status - {status}")
    else:
        print(f"Unsuccessful S3 put_object response. Status - {status}")

Successful S3 put_object response. Status - 200


In [9]:
Race_data_types = Race_Transformed.dtypes
print(Race_data_types)
print('Number of Rows:', Race_Transformed.shape[0])

GEO_ID                              object
TOTAL_ONE_RACE_18_OVER               int64
WHITE_ALONE_18_OVER                  int64
BLACK_ALONE_18_OVER                  int64
NATIVE_AM_ALONE_18_OVER              int64
ASIAN_ALONE_18_OVER                  int64
HAWAIIN_PAC_ISLAND_ALONE_18_OVER     int64
OTHER_ALONE_18_OVER                  int64
dtype: object
Number of Rows: 25607


In [10]:
stats_white = pd.DataFrame(Race_Transformed['WHITE_ALONE_18_OVER'].describe()).T
stats_white

,count,mean,std,min,25%,50%,75%,max
WHITE_ALONE_18_OVER,25607.0,525.005389,377.354033,0.0,265.0,458.0,707.0,23626.0


In [11]:
stats_black = pd.DataFrame(Race_Transformed['BLACK_ALONE_18_OVER'].describe()).T
stats_black

,count,mean,std,min,25%,50%,75%,max
BLACK_ALONE_18_OVER,25607.0,68.644043,116.431777,0.0,11.0,29.0,77.0,2945.0


In [12]:
stats_native = pd.DataFrame(Race_Transformed['NATIVE_AM_ALONE_18_OVER'].describe()).T
stats_native

,count,mean,std,min,25%,50%,75%,max
NATIVE_AM_ALONE_18_OVER,25607.0,18.572812,22.250073,0.0,6.0,14.0,26.0,923.0


In [13]:
stats_asian = pd.DataFrame(Race_Transformed['ASIAN_ALONE_18_OVER'].describe()).T
stats_asian

,count,mean,std,min,25%,50%,75%,max
ASIAN_ALONE_18_OVER,25607.0,194.103448,263.105348,0.0,36.0,99.0,241.0,5901.0


In [14]:
stats_hawiian = pd.DataFrame(Race_Transformed['HAWAIIN_PAC_ISLAND_ALONE_18_OVER'].describe()).T
stats_hawiian

,count,mean,std,min,25%,50%,75%,max
HAWAIIN_PAC_ISLAND_ALONE_18_OVER,25607.0,4.719413,9.175017,0.0,0.0,2.0,6.0,364.0


In [15]:
stats_other = pd.DataFrame(Race_Transformed['OTHER_ALONE_18_OVER'].describe()).T
stats_other

,count,mean,std,min,25%,50%,75%,max
OTHER_ALONE_18_OVER,25607.0,236.79447,241.131434,0.0,54.0,152.0,357.0,4249.0


In [16]:
s3_client = boto3.client("s3")
BUCKET='ads508project'
KEY='Athena_Data/Source_Files/Language_Spoken_At_Home.csv'
response = s3_client.get_object(Bucket=BUCKET, Key=KEY)
Language_Spoken = pd.read_csv(response.get("Body"))

TOTAL_LANGUAGE_SPOKEN_18_OVER = Language_Spoken["B16004_024E"] + Language_Spoken["B16004_046E"]
Language_Spoken["TOTAL_LANGUAGE_SPOKEN_18_OVER"] = TOTAL_LANGUAGE_SPOKEN_18_OVER

TOTAL_ENGLISH_NOT_AT_ALL_18_OVER = Language_Spoken["B16004_040E"] + Language_Spoken["B16004_035E"] + Language_Spoken["B16004_045E"] + Language_Spoken["B16004_030E"] + Language_Spoken["B16004_062E"] + Language_Spoken["B16004_057E"] + Language_Spoken["B16004_067E"] + Language_Spoken["B16004_052E"]
Language_Spoken["TOTAL_ENGLISH_NOT_AT_ALL_18_OVER"] = TOTAL_ENGLISH_NOT_AT_ALL_18_OVER

TOTAL_ENGLISH_NOT_WELL_18_OVER = Language_Spoken["B16004_039E"] + Language_Spoken["B16004_034E"] + Language_Spoken["B16004_044E"] + Language_Spoken["B16004_029E"] + Language_Spoken["B16004_061E"] + Language_Spoken["B16004_056E"] + Language_Spoken["B16004_066E"] + Language_Spoken["B16004_051E"]
Language_Spoken["TOTAL_ENGLISH_NOT_WELL_18_OVER"] = TOTAL_ENGLISH_NOT_WELL_18_OVER

TOTAL_ENGLISH_WELL_18_OVER = Language_Spoken["B16004_038E"] + Language_Spoken["B16004_033E"] + Language_Spoken["B16004_043E"] + Language_Spoken["B16004_028E"] + Language_Spoken["B16004_060E"] + Language_Spoken["B16004_055E"] + Language_Spoken["B16004_065E"] + Language_Spoken["B16004_050E"]
Language_Spoken["TOTAL_ENGLISH_WELL_18_OVER"] = TOTAL_ENGLISH_WELL_18_OVER

TOTAL_ENGLISH_VERY_WELL_18_OVER = Language_Spoken["B16004_037E"] + Language_Spoken["B16004_032E"] + Language_Spoken["B16004_042E"] + Language_Spoken["B16004_027E"] + Language_Spoken["B16004_059E"] + Language_Spoken["B16004_054E"] + Language_Spoken["B16004_064E"] + Language_Spoken["B16004_049E"]
Language_Spoken["TOTAL_ENGLISH_VERY_WELL_18_OVER"] = TOTAL_ENGLISH_VERY_WELL_18_OVER

TOTAL_ENGLISH_ONLY_18_OVER = Language_Spoken["B16004_025E"] + Language_Spoken["B16004_047E"]
Language_Spoken["TOTAL_ENGLISH_ONLY_18_OVER"] = TOTAL_ENGLISH_ONLY_18_OVER

Language_Spoken_Transformed = Language_Spoken[["GEO_ID", "TOTAL_LANGUAGE_SPOKEN_18_OVER", "TOTAL_ENGLISH_NOT_AT_ALL_18_OVER", "TOTAL_ENGLISH_NOT_WELL_18_OVER", "TOTAL_ENGLISH_WELL_18_OVER", "TOTAL_ENGLISH_VERY_WELL_18_OVER", "TOTAL_ENGLISH_ONLY_18_OVER"]]

Language_Spoken_Transformed.head()

,GEO_ID,TOTAL_LANGUAGE_SPOKEN_18_OVER,TOTAL_ENGLISH_NOT_AT_ALL_18_OVER,TOTAL_ENGLISH_NOT_WELL_18_OVER,TOTAL_ENGLISH_WELL_18_OVER,TOTAL_ENGLISH_VERY_WELL_18_OVER,TOTAL_ENGLISH_ONLY_18_OVER
0,1500000US060014001001,1320,0,0,0,238,1082
1,1500000US060014001002,1149,0,0,66,268,815
2,1500000US060014002001,754,0,4,11,70,669
3,1500000US060014002002,905,0,20,23,116,746
4,1500000US060014003001,927,0,0,0,108,819


In [17]:
s3_client = boto3.client("s3")
BUCKET='ads508project'
KEY='Athena_Data/Language_Spoken_At_Home/Language_Spoken_At_Home.csv'
response = s3_client.get_object(Bucket=BUCKET, Key=KEY)

with io.StringIO() as csv_buffer:
    Language_Spoken_Transformed.to_csv(csv_buffer, index=False)

    response = s3_client.put_object(
        Bucket=BUCKET, Key=KEY, Body=csv_buffer.getvalue()
    )

    status = response.get("ResponseMetadata", {}).get("HTTPStatusCode")

    if status == 200:
        print(f"Successful S3 put_object response. Status - {status}")
    else:
        print(f"Unsuccessful S3 put_object response. Status - {status}")

Successful S3 put_object response. Status - 200


In [18]:
Language_Spoken_data_types = Language_Spoken_Transformed.dtypes
print(Language_Spoken_data_types)
print('Number of Rows:', Language_Spoken_Transformed.shape[0])

GEO_ID                              object
TOTAL_LANGUAGE_SPOKEN_18_OVER        int64
TOTAL_ENGLISH_NOT_AT_ALL_18_OVER     int64
TOTAL_ENGLISH_NOT_WELL_18_OVER       int64
TOTAL_ENGLISH_WELL_18_OVER           int64
TOTAL_ENGLISH_VERY_WELL_18_OVER      int64
TOTAL_ENGLISH_ONLY_18_OVER           int64
dtype: object
Number of Rows: 25607


In [19]:
stats_no_eng = pd.DataFrame(Language_Spoken_Transformed['TOTAL_ENGLISH_NOT_AT_ALL_18_OVER'].describe()).T
stats_no_eng

,count,mean,std,min,25%,50%,75%,max
TOTAL_ENGLISH_NOT_AT_ALL_18_OVER,25607.0,41.01648,75.195247,0.0,0.0,8.0,51.0,1169.0


In [20]:
stats_eng_not_well = pd.DataFrame(Language_Spoken_Transformed['TOTAL_ENGLISH_NOT_WELL_18_OVER'].describe()).T
stats_eng_not_well

,count,mean,std,min,25%,50%,75%,max
TOTAL_ENGLISH_NOT_WELL_18_OVER,25607.0,86.373726,107.647551,0.0,8.0,47.0,126.0,972.0


In [21]:
stats_eng_well = pd.DataFrame(Language_Spoken_Transformed['TOTAL_ENGLISH_WELL_18_OVER'].describe()).T
stats_eng_well

,count,mean,std,min,25%,50%,75%,max
TOTAL_ENGLISH_WELL_18_OVER,25607.0,103.793806,105.035158,0.0,25.0,74.0,151.0,1126.0


In [22]:
stats_eng_very_well = pd.DataFrame(Language_Spoken_Transformed['TOTAL_ENGLISH_VERY_WELL_18_OVER'].describe()).T
stats_eng_very_well

,count,mean,std,min,25%,50%,75%,max
TOTAL_ENGLISH_VERY_WELL_18_OVER,25607.0,292.887726,238.835565,0.0,119.0,241.0,409.5,6288.0


In [23]:
stats_eng_only = pd.DataFrame(Language_Spoken_Transformed['TOTAL_ENGLISH_ONLY_18_OVER'].describe()).T
stats_eng_only

,count,mean,std,min,25%,50%,75%,max
TOTAL_ENGLISH_ONLY_18_OVER,25607.0,662.688991,456.92548,0.0,360.0,590.0,877.0,26430.0


In [24]:
s3_client = boto3.client("s3")
BUCKET='ads508project'
KEY='Athena_Data/Source_Files/HHIncome.csv'
response = s3_client.get_object(Bucket=BUCKET, Key=KEY)
HHIncome = pd.read_csv(response.get("Body"))

HHIncome_Transformed = HHIncome[["GEO_ID", "B19001_001E", "B19001_002E", "B19001_003E", "B19001_004E", "B19001_005E", "B19001_006E", "B19001_007E", "B19001_008E", "B19001_009E", "B19001_010E", "B19001_011E", "B19001_012E", "B19001_013E", "B19001_014E", "B19001_015E", "B19001_016E", "B19001_017E"]]

HHIncome_Transformed = HHIncome_Transformed.rename(columns={"B19001_001E":"TOTAL_HHINCOME", "B19001_002E":"LESS_THAN_10K", "B19001_003E":"10K_TO_14999", "B19001_004E":"15K_TO_19999", "B19001_005E":"20K_TO_24999", "B19001_006E":"25K_TO_29999", "B19001_007E":"30K_TO_34999", "B19001_008E":"35K_TO_39999", "B19001_009E":"40K_TO_44999", "B19001_010E":"45K_TO_49999", "B19001_011E":"50K_TO_59999", "B19001_012E":"60K_TO_74999", "B19001_013E":"75K_TO_99999", "B19001_014E":"100K_TO_124999", "B19001_015E":"125K_TO_149999", "B19001_016E":"150K_TO_199999", "B19001_017E":"200K_OR_MORE"}) 

HHIncome_Transformed.head()

,GEO_ID,TOTAL_HHINCOME,LESS_THAN_10K,10K_TO_14999,15K_TO_19999,20K_TO_24999,25K_TO_29999,30K_TO_34999,35K_TO_39999,40K_TO_44999,45K_TO_49999,50K_TO_59999,60K_TO_74999,75K_TO_99999,100K_TO_124999,125K_TO_149999,150K_TO_199999,200K_OR_MORE
0,1500000US060014001001,644,10,0,52,13,6,7,0,0,9,9,8,60,67,38,35,330
1,1500000US060014001002,630,37,6,0,6,7,0,7,0,8,23,9,7,25,54,47,394
2,1500000US060014002001,417,0,6,0,0,0,4,0,5,7,6,10,21,41,6,68,243
3,1500000US060014002002,413,0,5,0,0,0,0,23,0,0,12,27,37,44,38,54,173
4,1500000US060014003001,423,0,0,0,0,13,0,0,38,0,0,16,29,6,0,71,250


In [25]:
s3_client = boto3.client("s3")
BUCKET='ads508project'
KEY='Athena_Data/HHIncome/HHIncome.csv'
response = s3_client.get_object(Bucket=BUCKET, Key=KEY)

with io.StringIO() as csv_buffer:
    HHIncome_Transformed.to_csv(csv_buffer, index=False)

    response = s3_client.put_object(
        Bucket=BUCKET, Key=KEY, Body=csv_buffer.getvalue()
    )

    status = response.get("ResponseMetadata", {}).get("HTTPStatusCode")

    if status == 200:
        print(f"Successful S3 put_object response. Status - {status}")
    else:
        print(f"Unsuccessful S3 put_object response. Status - {status}")

Successful S3 put_object response. Status - 200


In [26]:
HHIncome_data_types = HHIncome_Transformed.dtypes
print(HHIncome_data_types)
print('Number of Rows:', HHIncome_Transformed.shape[0])

GEO_ID            object
TOTAL_HHINCOME     int64
LESS_THAN_10K      int64
10K_TO_14999       int64
15K_TO_19999       int64
20K_TO_24999       int64
25K_TO_29999       int64
30K_TO_34999       int64
35K_TO_39999       int64
40K_TO_44999       int64
45K_TO_49999       int64
50K_TO_59999       int64
60K_TO_74999       int64
75K_TO_99999       int64
100K_TO_124999     int64
125K_TO_149999     int64
150K_TO_199999     int64
200K_OR_MORE       int64
dtype: object
Number of Rows: 25607


In [27]:
stats_1 = pd.DataFrame(HHIncome_Transformed['10K_TO_14999'].describe()).T
stats_1

,count,mean,std,min,25%,50%,75%,max
10K_TO_14999,25607.0,19.814816,31.822168,0.0,0.0,8.0,27.0,468.0


In [28]:
stats_2 = pd.DataFrame(HHIncome_Transformed['15K_TO_19999'].describe()).T
stats_2

,count,mean,std,min,25%,50%,75%,max
15K_TO_19999,25607.0,17.00246,25.491078,0.0,0.0,8.0,25.0,270.0


In [29]:
stats_3 = pd.DataFrame(HHIncome_Transformed['20K_TO_24999'].describe()).T
stats_3

,count,mean,std,min,25%,50%,75%,max
20K_TO_24999,25607.0,18.514195,26.488879,0.0,0.0,9.0,27.0,354.0


In [30]:
stats_4 = pd.DataFrame(HHIncome_Transformed['25K_TO_29999'].describe()).T
stats_4

,count,mean,std,min,25%,50%,75%,max
25K_TO_29999,25607.0,17.744093,25.198738,0.0,0.0,9.0,26.0,585.0


In [31]:
stats_5 = pd.DataFrame(HHIncome_Transformed['30K_TO_34999'].describe()).T
stats_5

,count,mean,std,min,25%,50%,75%,max
30K_TO_34999,25607.0,18.56301,25.863688,0.0,0.0,10.0,27.0,477.0


In [32]:
stats_6 = pd.DataFrame(HHIncome_Transformed['40K_TO_44999'].describe()).T
stats_6

,count,mean,std,min,25%,50%,75%,max
40K_TO_44999,25607.0,18.040145,25.315658,0.0,0.0,10.0,27.0,549.0


In [33]:
stats_7 = pd.DataFrame(HHIncome_Transformed['45K_TO_49999'].describe()).T
stats_7

,count,mean,std,min,25%,50%,75%,max
45K_TO_49999,25607.0,16.168079,22.920819,0.0,0.0,9.0,24.0,431.0


In [34]:
stats_8 = pd.DataFrame(HHIncome_Transformed['50K_TO_59999'].describe()).T
stats_8

,count,mean,std,min,25%,50%,75%,max
50K_TO_59999,25607.0,32.992619,34.734195,0.0,8.0,24.0,48.0,792.0


In [35]:
stats_9 = pd.DataFrame(HHIncome_Transformed['60K_TO_74999'].describe()).T
stats_9

,count,mean,std,min,25%,50%,75%,max
60K_TO_74999,25607.0,45.404811,42.510665,0.0,15.0,35.0,64.0,886.0


In [36]:
stats_10 = pd.DataFrame(HHIncome_Transformed['75K_TO_99999'].describe()).T
stats_10

,count,mean,std,min,25%,50%,75%,max
75K_TO_99999,25607.0,63.120943,51.731508,0.0,26.0,52.0,87.0,1207.0


In [37]:
stats_11 = pd.DataFrame(HHIncome_Transformed['100K_TO_124999'].describe()).T
stats_11

,count,mean,std,min,25%,50%,75%,max
100K_TO_124999,25607.0,50.575624,45.730554,0.0,18.0,40.0,71.0,818.0


In [38]:
stats_12 = pd.DataFrame(HHIncome_Transformed['125K_TO_149999'].describe()).T
stats_12

,count,mean,std,min,25%,50%,75%,max
125K_TO_149999,25607.0,36.70965,38.874886,0.0,9.0,27.0,53.0,567.0


In [39]:
stats_13 = pd.DataFrame(HHIncome_Transformed['150K_TO_199999'].describe()).T
stats_13

,count,mean,std,min,25%,50%,75%,max
150K_TO_199999,25607.0,47.925333,49.904776,0.0,11.0,35.0,69.0,698.0


In [40]:
stats_14 = pd.DataFrame(HHIncome_Transformed['200K_OR_MORE'].describe()).T
stats_14

,count,mean,std,min,25%,50%,75%,max
200K_OR_MORE,25607.0,67.925762,88.975977,0.0,7.0,35.0,96.0,1419.0


In [41]:
s3_client = boto3.client("s3")
BUCKET='ads508project'
KEY='Athena_Data/Source_Files/Educational_Attainment.csv'
response = s3_client.get_object(Bucket=BUCKET, Key=KEY)
ED_Attainment = pd.read_csv(response.get("Body"))

SUM_ED_ATTAINMENT_COLUMNS = ED_Attainment["B15003_019E"] + ED_Attainment["B15003_020E"] + ED_Attainment["B15003_021E"] + ED_Attainment["B15003_022E"] + ED_Attainment["B15003_023E"] + ED_Attainment["B15003_024E"] + ED_Attainment["B15003_025E"]
ED_Attainment["TOTAL_GREATER_THAN_HS"] = SUM_ED_ATTAINMENT_COLUMNS

ED_Attainment_Transformed = ED_Attainment[["GEO_ID", "B15003_001E", "B15003_019E", "B15003_020E", "B15003_021E", "B15003_022E", "B15003_023E", "B15003_024E", "B15003_025E", "TOTAL_GREATER_THAN_HS"]]

ED_Attainment_Transformed = ED_Attainment_Transformed.rename(columns={"B15003_001E":"TOTAL_ED_ATTAINMENT", "B15003_019E":"SOME_COLLEGE_LESS_THAN_1_YEAR", "B15003_020E":"SOME_COLLEGE_1_YEAR_OR_MORE", "B15003_021E":"AA_DEGREE", "B15003_022E":"BA_DEGREE", "B15003_023E":"MA_DEGREE", "B15003_024E":"PROFESSIONAL_SCHOOL", "B15003_025E":"PHD"})

ED_Attainment_Transformed.head()

,GEO_ID,TOTAL_ED_ATTAINMENT,SOME_COLLEGE_LESS_THAN_1_YEAR,SOME_COLLEGE_1_YEAR_OR_MORE,AA_DEGREE,BA_DEGREE,MA_DEGREE,PROFESSIONAL_SCHOOL,PHD,TOTAL_GREATER_THAN_HS
0,1500000US060014001001,1277,5,75,8,472,403,125,137,1225
1,1500000US060014001002,1135,13,46,21,430,316,125,104,1055
2,1500000US060014002001,731,11,11,18,281,204,142,58,725
3,1500000US060014002002,874,8,57,21,264,317,36,72,775
4,1500000US060014003001,905,0,41,3,381,304,90,27,846


In [42]:
s3_client = boto3.client("s3")
BUCKET='ads508project'
KEY='Athena_Data/Educational_Attainment/Educational_Attainment.csv'
response = s3_client.get_object(Bucket=BUCKET, Key=KEY)

with io.StringIO() as csv_buffer:
    ED_Attainment_Transformed.to_csv(csv_buffer, index=False)

    response = s3_client.put_object(
        Bucket=BUCKET, Key=KEY, Body=csv_buffer.getvalue()
    )

    status = response.get("ResponseMetadata", {}).get("HTTPStatusCode")

    if status == 200:
        print(f"Successful S3 put_object response. Status - {status}")
    else:
        print(f"Unsuccessful S3 put_object response. Status - {status}")

Successful S3 put_object response. Status - 200


In [43]:
ED_Attainment_Transformed_data_types = ED_Attainment_Transformed.dtypes
print(ED_Attainment_Transformed_data_types)
print('Number of Rows:', ED_Attainment_Transformed.shape[0])

GEO_ID                           object
TOTAL_ED_ATTAINMENT               int64
SOME_COLLEGE_LESS_THAN_1_YEAR     int64
SOME_COLLEGE_1_YEAR_OR_MORE       int64
AA_DEGREE                         int64
BA_DEGREE                         int64
MA_DEGREE                         int64
PROFESSIONAL_SCHOOL               int64
PHD                               int64
TOTAL_GREATER_THAN_HS             int64
dtype: object
Number of Rows: 25607


In [44]:
stats_some_college1 = pd.DataFrame(ED_Attainment_Transformed['SOME_COLLEGE_LESS_THAN_1_YEAR'].describe()).T
stats_some_college1

,count,mean,std,min,25%,50%,75%,max
SOME_COLLEGE_LESS_THAN_1_YEAR,25607.0,60.857812,57.449131,0.0,21.0,47.0,85.0,1905.0


In [45]:
stats_some_college2 = pd.DataFrame(ED_Attainment_Transformed['SOME_COLLEGE_1_YEAR_OR_MORE'].describe()).T
stats_some_college2

,count,mean,std,min,25%,50%,75%,max
SOME_COLLEGE_1_YEAR_OR_MORE,25607.0,156.524935,110.835745,0.0,79.0,135.0,209.0,2470.0


In [46]:
stats_aa = pd.DataFrame(ED_Attainment_Transformed['AA_DEGREE'].describe()).T
stats_aa

,count,mean,std,min,25%,50%,75%,max
AA_DEGREE,25607.0,82.939313,68.792233,0.0,34.0,67.0,114.0,1274.0


In [47]:
stats_ba = pd.DataFrame(ED_Attainment_Transformed['BA_DEGREE'].describe()).T
stats_ba

,count,mean,std,min,25%,50%,75%,max
BA_DEGREE,25607.0,225.126997,180.707691,0.0,89.0,183.0,315.0,2207.0


In [48]:
stats_ma = pd.DataFrame(ED_Attainment_Transformed['MA_DEGREE'].describe()).T
stats_ma

,count,mean,std,min,25%,50%,75%,max
MA_DEGREE,25607.0,92.833093,104.491478,0.0,20.0,61.0,131.0,1768.0


In [49]:
stats_prof = pd.DataFrame(ED_Attainment_Transformed['PROFESSIONAL_SCHOOL'].describe()).T
stats_prof

,count,mean,std,min,25%,50%,75%,max
PROFESSIONAL_SCHOOL,25607.0,26.079978,40.848277,0.0,0.0,11.0,36.0,507.0


In [50]:
stats_phd = pd.DataFrame(ED_Attainment_Transformed['PHD'].describe()).T
stats_phd

,count,mean,std,min,25%,50%,75%,max
PHD,25607.0,17.457687,32.711118,0.0,0.0,0.0,22.0,734.0


In [51]:
stats_greater_than_hs = pd.DataFrame(ED_Attainment_Transformed['TOTAL_GREATER_THAN_HS'].describe()).T
stats_greater_than_hs

,count,mean,std,min,25%,50%,75%,max
TOTAL_GREATER_THAN_HS,25607.0,661.819815,391.190762,0.0,383.0,592.0,871.0,8351.0


In [52]:
pip install pyathena

/opt/conda/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
You should consider upgrading via the '/opt/conda/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [53]:
from pyathena import connect

In [54]:
sess = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

In [55]:
ingest_create_athena_db_passed = False

In [56]:
database_name = "ADS_508_Census_Data_Transformed"

In [57]:
s3_staging_dir = "s3://{0}/athena/staging".format(bucket)

In [58]:
conn = connect(region_name=region, s3_staging_dir=s3_staging_dir)

In [59]:
statement = "CREATE DATABASE IF NOT EXISTS {}".format(database_name)
print(statement)

CREATE DATABASE IF NOT EXISTS ADS_508_Census_Data_Transformed


In [60]:
census_data_dir = 's3://ads508project/Athena_Data'

In [61]:
# SQL statement to execute the analyte tests drinking water table

table_name ='Educational_Attainment'
pd.read_sql(f'DROP TABLE IF EXISTS {database_name}.{table_name}', conn)


create_table = f"""
CREATE EXTERNAL TABLE IF NOT EXISTS {database_name}.{table_name}(
                GEO_ID string,
                TOTAL_ED_ATTAINMENT int,
                SOME_COLLEGE_LESS_THAN_1_YEAR int,
                SOME_COLLEGE_1_YEAR_OR_MORE int,
                AA_DEGREE int,
                BA_DEGREE int,
                MA_DEGREE int,
                PROFESSIONAL_SCHOOL int,
                PHD int,
                TOTAL_GREATER_THAN_HS int
                )
                
                ROW FORMAT DELIMITED
                FIELDS TERMINATED BY ','
                LOCATION '{census_data_dir}/{table_name}'
                TBLPROPERTIES ('skip.header.line.count'='1')
"""

pd.read_sql(create_table, conn)

pd.read_sql(f'SELECT * FROM {database_name}.{table_name} LIMIT 5', conn)

,geo_id,total_ed_attainment,some_college_less_than_1_year,some_college_1_year_or_more,aa_degree,ba_degree,ma_degree,professional_school,phd,total_greater_than_hs
0,1500000US060014001001,1277,5,75,8,472,403,125,137,1225
1,1500000US060014001002,1135,13,46,21,430,316,125,104,1055
2,1500000US060014002001,731,11,11,18,281,204,142,58,725
3,1500000US060014002002,874,8,57,21,264,317,36,72,775
4,1500000US060014003001,905,0,41,3,381,304,90,27,846


In [62]:
# SQL statement to execute the analyte tests drinking water table

table_name ='Ethnicity_Over_18'
pd.read_sql(f'DROP TABLE IF EXISTS {database_name}.{table_name}', conn)


create_table = f"""
CREATE EXTERNAL TABLE IF NOT EXISTS {database_name}.{table_name}(
                GEO_ID string,
                TOTAL_ETHNICITY_18_OVER int,
                HISP_LATINO_18_OVER int,
                NOT_HISP_LATINO_18_OVER int
                )
                
                ROW FORMAT DELIMITED
                FIELDS TERMINATED BY ','
                LOCATION '{census_data_dir}/{table_name}'
                TBLPROPERTIES ('skip.header.line.count'='1')
"""

pd.read_sql(create_table, conn)

pd.read_sql(f'SELECT * FROM {database_name}.{table_name} LIMIT 5', conn)

,geo_id,total_ethnicity_18_over,hisp_latino_18_over,not_hisp_latino_18_over
0,1500000US060014001001,1540,92,1448
1,1500000US060014001002,1093,57,1036
2,1500000US060014002001,832,74,758
3,1500000US060014002002,820,73,747
4,1500000US060014003001,914,62,852


In [63]:
# SQL statement to execute the analyte tests drinking water table

table_name ='Race_Over_18'
pd.read_sql(f'DROP TABLE IF EXISTS {database_name}.{table_name}', conn)


create_table = f"""
CREATE EXTERNAL TABLE IF NOT EXISTS {database_name}.{table_name}(
                GEO_ID string,
                TOTAL_ONE_RACE_18_OVER int,
                WHITE_ALONE_18_OVER int,
                BLACK_ALONE_18_OVER int,
                NATIVE_AM_ALONE_18_OVER int,
                ASIAN_ALONE_18_OVER int,
                HAWAIIN_PAC_ISLAND_ALONE_18_OVER int,
                OTHER_ALONE_18_OVER int
                )
                
                ROW FORMAT DELIMITED
                FIELDS TERMINATED BY ','
                LOCATION '{census_data_dir}/{table_name}'
                TBLPROPERTIES ('skip.header.line.count'='1')
"""

pd.read_sql(create_table, conn)

pd.read_sql(f'SELECT * FROM {database_name}.{table_name} LIMIT 5', conn)

,geo_id,total_one_race_18_over,white_alone_18_over,black_alone_18_over,native_am_alone_18_over,asian_alone_18_over,hawaiin_pac_island_alone_18_over,other_alone_18_over
0,1500000US060014001001,1377,1067,42,2,223,6,37
1,1500000US060014001002,996,638,77,0,265,2,14
2,1500000US060014002001,754,642,12,0,74,2,24
3,1500000US060014002002,735,580,27,0,114,0,14
4,1500000US060014003001,838,683,29,2,110,0,14


In [64]:
# SQL statement to execute the analyte tests drinking water table

table_name ='Language_Spoken_At_Home'
pd.read_sql(f'DROP TABLE IF EXISTS {database_name}.{table_name}', conn)


create_table = f"""
CREATE EXTERNAL TABLE IF NOT EXISTS {database_name}.{table_name}(
                GEO_ID string,
                TOTAL_LANGUAGE_SPOKEN_18_OVER int,
                TOTAL_ENGLISH_NOT_AT_ALL_18_OVER int,
                TOTAL_ENGLISH_NOT_WELL_18_OVER int,
                TOTAL_ENGLISH_WELL_18_OVER int,
                TOTAL_ENGLISH_VERY_WELL_18_OVER int,
                TOTAL_ENGLISH_ONLY_18_OVER int
                )
                
                ROW FORMAT DELIMITED
                FIELDS TERMINATED BY ','
                LOCATION '{census_data_dir}/{table_name}'
                TBLPROPERTIES ('skip.header.line.count'='1')
"""

pd.read_sql(create_table, conn)

pd.read_sql(f'SELECT * FROM {database_name}.{table_name} LIMIT 5', conn)

,geo_id,total_language_spoken_18_over,total_english_not_at_all_18_over,total_english_not_well_18_over,total_english_well_18_over,total_english_very_well_18_over,total_english_only_18_over
0,1500000US060014001001,1320,0,0,0,238,1082
1,1500000US060014001002,1149,0,0,66,268,815
2,1500000US060014002001,754,0,4,11,70,669
3,1500000US060014002002,905,0,20,23,116,746
4,1500000US060014003001,927,0,0,0,108,819


In [65]:
# SQL statement to execute the analyte tests drinking water table

table_name ='HHIncome'
pd.read_sql(f'DROP TABLE IF EXISTS {database_name}.{table_name}', conn)


create_table = f"""
CREATE EXTERNAL TABLE IF NOT EXISTS {database_name}.{table_name}(
                GEO_ID string,
                TOTAL_HHINCOME int,
                LESS_THAN_10K int,
                10K_TO_14999 int,
                15K_TO_19999 int,
                20K_TO_24999 int,
                25K_TO_29999 int,
                30K_TO_34999 int,
                35K_TO_39999 int,
                40K_TO_44999 int,
                45K_TO_49999 int,
                50K_TO_59999 int,
                60K_TO_74999 int,
                75K_TO_99999 int,
                100K_TO_124999 int,
                125K_TO_149999 int,
                150K_TO_199999 int,
                200K_OR_MORE int
                )
                
                ROW FORMAT DELIMITED
                FIELDS TERMINATED BY ','
                LOCATION '{census_data_dir}/{table_name}'
                TBLPROPERTIES ('skip.header.line.count'='1')
"""

pd.read_sql(create_table, conn)

pd.read_sql(f'SELECT * FROM {database_name}.{table_name} LIMIT 5', conn)

,geo_id,total_hhincome,less_than_10k,10k_to_14999,15k_to_19999,20k_to_24999,25k_to_29999,30k_to_34999,35k_to_39999,40k_to_44999,45k_to_49999,50k_to_59999,60k_to_74999,75k_to_99999,100k_to_124999,125k_to_149999,150k_to_199999,200k_or_more
0,1500000US060014001001,644,10,0,52,13,6,7,0,0,9,9,8,60,67,38,35,330
1,1500000US060014001002,630,37,6,0,6,7,0,7,0,8,23,9,7,25,54,47,394
2,1500000US060014002001,417,0,6,0,0,0,4,0,5,7,6,10,21,41,6,68,243
3,1500000US060014002002,413,0,5,0,0,0,0,23,0,0,12,27,37,44,38,54,173
4,1500000US060014003001,423,0,0,0,0,13,0,0,38,0,0,16,29,6,0,71,250


In [66]:
Merged_Census_Data = pd.read_sql(f'SELECT * FROM {database_name}.Educational_Attainment left outer join {database_name}.HHIncome on (Educational_Attainment.geo_id = HHIncome.geo_id) left outer join {database_name}.Language_Spoken_At_Home on (Educational_Attainment.geo_id = Language_Spoken_At_Home.geo_id) left outer join {database_name}.Race_Over_18 on (Educational_Attainment.geo_id = Race_Over_18.geo_id) left outer join {database_name}.Ethnicity_Over_18 on (Educational_Attainment.geo_id = Ethnicity_Over_18.geo_id)', conn)

In [67]:
Merged_Census_Data.head()

,geo_id,total_ed_attainment,some_college_less_than_1_year,some_college_1_year_or_more,aa_degree,ba_degree,ma_degree,professional_school,phd,total_greater_than_hs,...,white_alone_18_over,black_alone_18_over,native_am_alone_18_over,asian_alone_18_over,hawaiin_pac_island_alone_18_over,other_alone_18_over,geo_id,total_ethnicity_18_over,hisp_latino_18_over,not_hisp_latino_18_over
0,1500000US060014001001,1277,5,75,8,472,403,125,137,1225,...,1067,42,2,223,6,37,1500000US060014001001,1540,92,1448
1,1500000US060014001002,1135,13,46,21,430,316,125,104,1055,...,638,77,0,265,2,14,1500000US060014001002,1093,57,1036
2,1500000US060014002001,731,11,11,18,281,204,142,58,725,...,642,12,0,74,2,24,1500000US060014002001,832,74,758
3,1500000US060014002002,874,8,57,21,264,317,36,72,775,...,580,27,0,114,0,14,1500000US060014002002,820,73,747
4,1500000US060014003001,905,0,41,3,381,304,90,27,846,...,683,29,2,110,0,14,1500000US060014003001,914,62,852


In [70]:
print(Merged_Census_Data.shape)

(25607, 47)
